In [1]:
import os
import chardet
from pathlib import Path
import pyeeg

import mne
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import r2_score
from sklearn.metrics import confusion_matrix
from sklearn.svm import SVC

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

import matplotlib.pyplot as plt

Worked on after eeg_motor_imagery_nn.ipynb to extract various features and then model. 

Changes needed:
edit 27th July, 2022
1. for each of the epochs break into various bands [break into 5 bands alpha, beta...] start calculating the features
2. features: psd, attention (alpha band/beta band), frontal assymetery (difference between the left hemisphere and right)
3. average of time_series_data (just take the average of epoch)


Sources used:
https://mne.tools/stable/generated/mne.time_frequency.psd_welch.html (visited: 05/08/2022)
https://mne.tools/stable/auto_tutorials/time-freq/20_sensors_time_frequency.html#sphx-glr-auto-tutorials-time-freq-20-sensors-time-frequency-py (visited: 05/08/2022)

In [2]:
def load_subjects_task(subject, task, run_number_left = 0, run_number_right = 3):
    event_id = dict(hands=2, feet=3)
    runs = [[3, 7, 11], [4, 8, 12], [5, 9, 13], [6, 10, 14]]   
    # runs[0] open and close left or right fist
    # runs[1] imagine opening and closing left or right fist
    # runs[2] open and close both fists or both feet
    # runs[3] imagine opening and closing both fists or both feet

    """
        T0 corresponds to rest
        T1 corresponds to onset of motion (real or imagined) of
            the left fist (in runs 3, 4, 7, 8, 11, and 12)
            both fists (in runs 5, 6, 9, 10, 13, and 14)
        T2 corresponds to onset of motion (real or imagined) of
            the right fist (in runs 3, 4, 7, 8, 11, and 12)
            both feet (in runs 5, 6, 9, 10, 13, and 14)
    """
    raw_fnames = mne.datasets.eegbci.load_data(subject, runs[task][run_number_left:run_number_right])
    raw = mne.io.concatenate_raws([mne.io.read_raw_edf(f, preload=True) for f in raw_fnames])
    channel_rename_dict = {'Fc5.':'FC5', 'Fc3.':'FC3', 'Fc1.':'FC1', 'Fcz.':'FCz', 
                      'Fc2.':'FC2', 'Fc4.':'FC4', 'Fc6.':'FC6', 'C5..':'C5', 
                      'C3..':'C3', 'C1..':'C1', 'Cz..':'Cz', 'C2..':'C2', 
                      'C4..':'C4', 'C6..':'C6', 'Cp5.':'CP5', 'Cp3.':'CP3', 
                      'Cp1.':'CP1', 'Cpz.':'CPz', 'Cp2.':'CP2', 'Cp4.':'CP4', 
                      'Cp6.':'CP6', 'Fp1.':'Fp1', 'Fpz.':'Fpz', 'Fp2.':'Fp2', 
                      'Af7.':'AF7', 'Af3.':'AF3', 'Afz.':'AFz', 'Af4.':'AF4', 
                      'Af8.':'AF8', 'F7..':'F7', 'F5..':'F5', 'F3..':'F3', 
                      'F1..':'F1', 'Fz..':'Fz', 'F2..':'F2', 'F4..':'F4', 
                      'F6..':'F6', 'F8..':'F8', 'Ft7.':'FT7', 'Ft8.':'FT8', 
                      'T7..':'T7', 'T8..':'T8', 'T9..':'T9', 'T10.':'T10', 
                      'Tp7.':'TP7', 'Tp8.':'TP8', 'P7..':'P7', 'P5..':'P5', 
                      'P3..':'P3', 'P1..':'P1', 'Pz..':'Pz', 'P2..':'P2', 
                      'P4..':'P4', 'P6..':'P6', 'P8..':'P8', 'Po7.':'PO7', 
                      'Po3.':'PO3', 'Poz.':'POz', 'Po4.':'PO4', 'Po8.':'PO8', 
                      'O1..':'O1', 'Oz..':'Oz', 'O2..':'O2', 'Iz..':'Iz'}
    raw.rename_channels(channel_rename_dict)
    montage = mne.channels.make_standard_montage('standard_1005')
    raw.set_montage(montage)
    events, _ = mne.events_from_annotations(raw, event_id=dict(T0=1, T1=2, T2=3))
    return raw, events

In [3]:
def fft_mean(data, fs, plotting = 0):
    """
    Parameters
    ----------
    data : is 1d list/np.array
        of the signal
    fs : int
        sampling frequency
    plotting : int | 0 or 1
        plots the eeg band values
    
    Returns
    -------
    eeg_band_fft : dict 
        mean of power of signal in that band
    """
    # data is 1d
    # Get real amplitudes of FFT (only in postive frequencies)
    fft_vals = np.absolute(np.fft.rfft(data))
    # Get frequencies for amplitudes in Hz
    fft_freq = np.fft.rfftfreq(len(data), 1.0/fs)
    # print(fft_vals.shape, fft_freq.shape)
    
    # Define EEG bands
    eeg_bands = {'Delta': (0, 4),
                 'Theta': (4, 8),
                 'Alpha': (8, 12),
                 'Beta': (12, 30),
                 'Gamma': (30, 45)}

    # Take the mean of the fft amplitude for each EEG band
    # TODO: doubt should I take mean of this or not
    eeg_band_fft = dict()
    for band in eeg_bands:  
        freq_ix = np.where((fft_freq >= eeg_bands[band][0]) & 
                           (fft_freq <= eeg_bands[band][1]))[0]
        eeg_band_fft[band] = np.mean(fft_vals[freq_ix])
        # eeg_band_fft[band] = fft_vals[freq_ix]
    
    # print("PRINTING EEG_BAND_FFT", eeg_band_fft)
    # Plot the data (using pandas here cause it's easy)
    if plotting == 1:
        df = pd.DataFrame(columns=['band', 'val'])
        df['band'] = eeg_bands.keys()
        df['val'] = [eeg_band_fft[band] for band in eeg_bands]
        for band in eeg_bands:
            print(eeg_band_fft[band])
        ax = df.plot.bar(x='band', y='val', legend=False)
        ax.set_xlabel("EEG band")
        ax.set_ylabel("Mean band Amplitude")
    return eeg_band_fft

In [4]:
def preprocess_raw_data_2d_extract_features(raw, events, participant_id, df_eeg_features, plotting=0):
    # edit 27th July, 2022
    # for each of the epochs break into various bands [break into 5 bands alpha, beta...]
    # start calculating the features
    # features: psd, attention (alpha band/beta band), frontal assymetery (difference between the left hemisphere and right)
    # average of time_series_data (just take the average of epoch)
    raw.filter(7., 30., fir_design='firwin', skip_by_annotation='edge')
    event_id = dict(hands=2, feet=3)
    tmin, tmax = -1., 4.
    picks = mne.pick_types(raw.info, meg=False, eeg=True, stim=False, eog=False)
    epochs = mne.Epochs(raw, events, event_id, tmin, tmax, proj=True, picks=picks,
                baseline=None, preload=True)
    epochs_train = epochs.copy().crop(tmin=1., tmax=2.)
    labels = epochs.events[:, -1] - 2
    i = 0
    for epoch in epochs_train:
        
        # bands
        eeg_band_fft = fft_mean(epoch, epochs.info['sfreq'])
        # print(eeg_band_fft)
        eeg_band_fft["label"] = labels[i]
        eeg_band_fft["participant_id"] = participant_id
        
        # epoch average
        epoch_avg = np.average(epoch)
        eeg_band_fft["epoch_avg"] = epoch_avg
        
        # psd
        # TODO: DOUBT how to add this to dataset
        # what should n_fft be: 161, 100, or 99
        kwargs = dict(fmin=2, fmax=40)
        psds_welch_mean, freqs_mean = mne.time_frequency.psd_welch(epochs_train[i], n_fft=99, average='mean', **kwargs)
        psds_welch_median, freqs_median = mne.time_frequency.psd_welch(epochs_train[i], n_fft=99, average='median', **kwargs)
        
        
        if plotting == 1:
            # We will only plot the PSD for a single sensor in the first epoch.
            ch_name = 'Fp1'
            ch_idx = epochs.info['ch_names'].index(ch_name)
            print("PRINTING CH_IDX", ch_idx)
            epo_idx = 0
            _, ax = plt.subplots()
            ax.plot(freqs_mean, psds_welch_mean[epo_idx, ch_idx, :], color='k',
                    ls='-', label='mean of segments')
            ax.plot(freqs_median, psds_welch_median[epo_idx, ch_idx, :], color='k',
                    ls='--', label='median of segments')

            ax.set(title='Welch PSD ({}, Epoch {})'.format(ch_name, epo_idx),
                   xlabel='Frequency (Hz)', ylabel='Power Spectral Density (dB)')
            ax.legend(loc='upper right')
            plt.show()

        
        df_eeg_features = df_eeg_features.append(eeg_band_fft, ignore_index=True)
        i += 1
    return df_eeg_features

In [5]:
def load_participant_data_extract_features(num_of_participants = 109):
    training_data_of_participants = []
    labels_of_participants = []
    df_eeg_features = pd.DataFrame()
    for i in range(1, num_of_participants + 1):
        subject_raw, subject_events = load_subjects_task(i, 0)
        df_eeg_features = preprocess_raw_data_2d_extract_features(subject_raw, subject_events, i, df_eeg_features, plotting = 0)
    return df_eeg_features

In [6]:
df_bands = load_participant_data_extract_features(30)

Extracting EDF parameters from /home/vanilla_skies/mne_data/MNE-eegbci-data/files/eegmmidb/1.0.0/S001/S001R03.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19999  =      0.000 ...   124.994 secs...
Extracting EDF parameters from /home/vanilla_skies/mne_data/MNE-eegbci-data/files/eegmmidb/1.0.0/S001/S001R07.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19999  =      0.000 ...   124.994 secs...
Extracting EDF parameters from /home/vanilla_skies/mne_data/MNE-eegbci-data/files/eegmmidb/1.0.0/S001/S001R11.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19999  =      0.000 ...   124.994 secs...
Used Annotations descriptions: ['T0', 'T1', 'T2']
Filtering raw data in 3 contiguous segments
Setting up band-pass filter from 7 - 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandp

/tmp/ipykernel_18001/1951054575.py:54: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_eeg_features = df_eeg_features.append(eeg_band_fft, ignore_index=True)
/tmp/ipykernel_18001/1951054575.py:54: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_eeg_features = df_eeg_features.append(eeg_band_fft, ignore_index=True)
/tmp/ipykernel_18001/1951054575.py:54: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_eeg_features = df_eeg_features.append(eeg_band_fft, ignore_index=True)


Effective window size : 0.619 (s)
Effective window size : 0.619 (s)
Effective window size : 0.619 (s)
Effective window size : 0.619 (s)
Effective window size : 0.619 (s)
Effective window size : 0.619 (s)


/tmp/ipykernel_18001/1951054575.py:54: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_eeg_features = df_eeg_features.append(eeg_band_fft, ignore_index=True)
/tmp/ipykernel_18001/1951054575.py:54: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_eeg_features = df_eeg_features.append(eeg_band_fft, ignore_index=True)
/tmp/ipykernel_18001/1951054575.py:54: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_eeg_features = df_eeg_features.append(eeg_band_fft, ignore_index=True)


Effective window size : 0.619 (s)
Effective window size : 0.619 (s)
Effective window size : 0.619 (s)
Effective window size : 0.619 (s)
Effective window size : 0.619 (s)
Effective window size : 0.619 (s)


...

Effective window size : 0.619 (s)
Effective window size : 0.619 (s)
Effective window size : 0.619 (s)
Effective window size : 0.619 (s)
Effective window size : 0.619 (s)
Effective window size : 0.619 (s)
Effective window size : 0.619 (s)
Effective window size : 0.619 (s)


/tmp/ipykernel_18001/1951054575.py:54: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_eeg_features = df_eeg_features.append(eeg_band_fft, ignore_index=True)
/tmp/ipykernel_18001/1951054575.py:54: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_eeg_features = df_eeg_features.append(eeg_band_fft, ignore_index=True)
/tmp/ipykernel_18001/1951054575.py:54: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_eeg_features = df_eeg_features.append(eeg_band_fft, ignore_index=True)
/tmp/ipykernel_18001/1951054575.py:54: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_eeg_features = df_eeg_features.append(eeg_band_fft, ignore_index=True)


Effective window size : 0.619 (s)
Effective window size : 0.619 (s)
Effective window size : 0.619 (s)
Effective window size : 0.619 (s)
Effective window size : 0.619 (s)
Effective window size : 0.619 (s)


/tmp/ipykernel_18001/1951054575.py:54: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_eeg_features = df_eeg_features.append(eeg_band_fft, ignore_index=True)
/tmp/ipykernel_18001/1951054575.py:54: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_eeg_features = df_eeg_features.append(eeg_band_fft, ignore_index=True)
/tmp/ipykernel_18001/1951054575.py:54: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_eeg_features = df_eeg_features.append(eeg_band_fft, ignore_index=True)
/tmp/ipykernel_18001/1951054575.py:54: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_eeg_features = df_eeg_features.append(eeg_band_fft, ignore_index=True)


SSLError: [SSL: DECRYPTION_FAILED_OR_BAD_RECORD_MAC] decryption failed or bad record mac (_ssl.c:2635)

In [ ]:
df_bands["label"] = df_bands["label"].astype('int')
df_bands

In [ ]:
df_bands_2 = df_bands.drop(["participant_id"], axis=1)

In [ ]:
X = df_bands_2[df_bands_2.columns.difference(["label"])]
y = df_bands_2["label"]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state=4, stratify=y)
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

In [ ]:
model = SVC()
model.fit(X_train, y_train)
predictions = model.predict(X_test)
print(list(predictions))
print(list(y_test))

In [ ]:
score_of_the_model = model.score(X_test, y_test)
print('model score: {}'.format(score_of_the_model))

In [ ]:
clf = make_pipeline(StandardScaler(), SVC(gamma='auto'))
clf.fit(X_train, y_train)
score_of_the_model = clf.score(X_test, y_test)
print('model score: {}'.format(score_of_the_model))